In [17]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [83]:
prc = pd.read_csv('../sf_parcels.csv.gz', usecols=['from_address_num', 'to_address_num', 'street_name', 'shape'])
prc['shape'] = prc['shape'].apply(wkt.loads)
prc = gpd.GeoDataFrame(prc, geometry='shape')

zn = pd.read_csv('../sf_zoning.csv.gz')
zn['the_geom'] = zn['the_geom'].apply(wkt.loads)
zn = gpd.GeoDataFrame(zn, geometry='the_geom')

prc

,from_address_num,to_address_num,street_name,shape
0,0.0,0.0,UNKNOWN,"MULTIPOLYGON (((-122.42200 37.80848, -122.4220..."
1,0.0,0.0,UNKNOWN,"MULTIPOLYGON (((-122.42083 37.80863, -122.4208..."
2,160.0,160.0,JEFFERSON,"MULTIPOLYGON (((-122.41570 37.80833, -122.4157..."
3,206.0,206.0,JEFFERSON,"MULTIPOLYGON (((-122.41735 37.80812, -122.4174..."
4,350.0,366.0,JEFFERSON,"MULTIPOLYGON (((-122.41897 37.80791, -122.4191..."
...,...,...,...,...
235048,2872.0,2888.0,JACKSON,"MULTIPOLYGON (((-122.44253 37.79155, -122.4425..."
235049,2872.0,2888.0,JACKSON,"MULTIPOLYGON (((-122.44253 37.79155, -122.4425..."
235050,2872.0,2888.0,JACKSON,"MULTIPOLYGON (((-122.44253 37.79155, -122.4425..."
235051,2872.0,2888.0,JACKSON,"MULTIPOLYGON (((-122.44253 37.79155, -122.4425..."


In [80]:
my_house = prc[(prc['street_name'] == 'NOE')
               & (prc['from_address_num'] <= 483)
               & (483 <= prc['to_address_num'])]

my_zoning = gpd.sjoin(my_house, zn, how="inner", predicate="intersects")
len(my_zoning)

1

In [69]:
from typing import List, Optional
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
import dotenv
dotenv.load_dotenv()

model = ChatOpenAI(model="gpt-3.5-turbo-1106")

class Address(BaseModel):
    number: str = Field(description="the house number of the address")
    street: str = Field(description="the street the address is on, in all caps, without any suffix like st or rd")

class AddressResponse(BaseModel):
    addresses: List[Address] = Field(description="all addresses found in the query")

template_str = """
Extract addresses from the following query.

{query}

{format_instructions}
"""

parser = PydanticOutputParser(pydantic_object=AddressResponse)
prompt = PromptTemplate(
    template=template_str,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

query = "What can I build at 483 Noe st?"
prompt_and_model = prompt | model
output = prompt_and_model.invoke(
    {
        "query": query,
    }
)
response = parser.invoke(output)

[a for a in response.addresses if a.number in query]

[Address(number='483', street='NOE')]